# `rioxarray`

`rioxarray`:

- is a Pythone extension for `xarray` to manipulate `xarray.DataArray` s as rasters
- means *Raster input/output + xarray *

## Data

Raster files from the US national ag imagery program (NAIP)

- high resolution aerial images from 2020 with four spectral bands: Red, Green, Blue, and Near infrared ( NIR)



In [ ]:
## import .tif